in this notebook we will follow the framework laid out by openai to finetune: [link](https://platform.openai.com/docs/guides/fine-tuning/introduction)

we will syntheitcally create our dataset using an LLM by boostrapping a few samples. we will then evaulate the validity of this dataset using an LLM.

will repurpose the Q&A on Retrieved Data in Arize


In [1]:
import os
import dotenv

import pandas as pd
import json
from jinja2 import Template
from sklearn.metrics import classification_report
from pycm import ConfusionMatrix
import matplotlib.pyplot as plt

from openai import OpenAI
import phoenix.evals.default_templates as templates
from phoenix.evals import (
    OpenAIModel,
    llm_classify,
)

import nest_asyncio

dotenv.load_dotenv()
nest_asyncio.apply()

/Users/rushilsheth/Documents/portfolio/fine-tuning/example_notebooks/openai-small-example-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# lets try to predict normally with 3.5-turbo

In [3]:
positive_normal = '''I recently purchased the Galaxy Explorer drone and am absolutely thrilled with its performance. The drone's battery life is impressive, allowing for extended flight times, and the camera quality is outstanding, capturing crisp and clear images from great heights. The intuitive controls made it easy for me as a beginner to navigate, and I've been able to capture some truly breathtaking footage. Highly recommend to anyone looking for a reliable and high-quality drone.'''
negative_normal = '''Had dinner at The Green Terrace last night and was deeply disappointed. Despite the cozy ambiance, the service was sluggish, and our orders took forever to arrive. When the food finally came, it was lukewarm at best. The pasta was overcooked, and the salad lacked freshness. It's a shame because I had high expectations based on the reviews. Sadly, I won't be returning or recommending this place to friends.'''
positive_tough = '''Lost my job, but at least I won't have to endure that dreadful commute anymore'''
negative_tough = '''Your presentation was surprisingly good; I expected much less'''

SYSTEM_PROMPT = '''You are given a social media review. Classify it as postive, negative. '''

original_dataset = [[positive_normal, 'positive'],
               [negative_normal, 'negative'], 
               [positive_tough, 'positive'],
               [negative_tough, 'negative']]

In [4]:
def classify_sentiment(social_media_post, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(model=model, 
                                              temperature = 0.1,
                                              messages=[{"role": "system", "content": SYSTEM_PROMPT}, 
                                                        {"role": "user", "content": social_media_post}])
    return response.choices[0].message.content

In [5]:
df = pd.DataFrame(original_dataset, columns=['text', 'label'])
df['predicted_label'] = df['text'].apply(classify_sentiment).apply(lambda x: x.strip().lower())

In [6]:
df['correct'] = df['label'] == df['predicted_label']

In [7]:
original_incorrect = df[~df['correct']]

## accuracy for base gpt-3.5 model

In [8]:
display(df)

,text,label,predicted_label,correct
0,I recently purchased the Galaxy Explorer drone...,positive,positive,True
1,Had dinner at The Green Terrace last night and...,negative,negative,True
2,"Lost my job, but at least I won't have to endu...",positive,negative,False
3,Your presentation was surprisingly good; I exp...,negative,positive,False


In [9]:
df['correct'].mean()

0.5

# build a dataset for finetuning

In [10]:
# Format the examples
formatted_examples = ",\n    ".join([f"'set-{i+1}': (\"{text}\", '{sentiment}')" for i, (text, sentiment) in enumerate(original_dataset)])
formatted_examples = '{'+formatted_examples+'}'

# Step 1: Read the file content
with open('dataset_generator_template.txt', 'r') as file:
    file_content = file.read()

# Step 3: Render the template with examples
template = Template(file_content)
dataset_gen_system_prompt = template.render(examples=formatted_examples)

print(dataset_gen_system_prompt)


You are responsible for creating a dataset of social media posts and their associated sentiments(postiive or negative).

Read the instructions below carefully and follow each step in order to accurately generate your response.

Please generate 20 sets of social media posts. Each set should not share similar context.

Please output the sets in the following format:
{'set-1': [<POST>, sentiment],
    'set-2': [<POST>, sentiment], ...,
    'set-20': [<POST>, sentiment]}

Here are some examples:
{'set-1': ("I recently purchased the Galaxy Explorer drone and am absolutely thrilled with its performance. The drone's battery life is impressive, allowing for extended flight times, and the camera quality is outstanding, capturing crisp and clear images from great heights. The intuitive controls made it easy for me as a beginner to navigate, and I've been able to capture some truly breathtaking footage. Highly recommend to anyone looking for a reliable and high-quality drone.", 'positive'),
    '

In [11]:
def generate_synthetic_dataset():
    dataset_response = client.chat.completions.create(model='gpt-4-turbo-preview', 
                                              temperature = 0.1,
                                              messages=[{"role": "system", "content": dataset_gen_system_prompt}, 
                                                        {"role": "user", "content": "JSON Response:"}]).choices[0].message.content
    # sometimes the string ```json``` is not present in the response, so we need to check for it
    # if it is in there we want to remove it and the trailing ``` from the response
    if 'json' in dataset_response:
        parsed_response = dataset_response.replace('json', '')
        parsed_response = parsed_response.replace('`', '')
        parsed_response = json.loads(parsed_response)
    else:
        parsed_response = json.loads(dataset_response)
    
    return parsed_response

In [12]:
parsed_response = generate_synthetic_dataset()

In [13]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(parsed_response, orient='index', columns=['reference', 'output'])
# add some inccorect ones to see how the judge LLM does
df.loc['set-21'] = [positive_normal, 'negative']
df.loc['set-22'] = [negative_normal, 'positive']
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)
df['input'] =  SYSTEM_PROMPT


## evalute dataset for fine tuning

throw some bad ones in there to ensure our eval is working well

In [14]:
print(templates.QA_PROMPT_TEMPLATE)


You are given a question, an answer and reference text. You must determine whether the
given answer correctly answers the question based on the reference text. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {input}
    ************
    [Reference]: {reference}
    ************
    [Answer]: {output}
    [END DATA]
Your response must be a single word, either "correct" or "incorrect",
and should not contain any text or characters aside from that word.
"correct" means that the question is correctly and fully answered by the answer.
"incorrect" means that the question is not correctly or only partially answered by the
answer.



### As you can see, the Judge LLM rightly identifies the incorrect training data

In [15]:
model = OpenAIModel(
    model="gpt-4",
    temperature=0.0,
)
rails = list(templates.QA_PROMPT_RAILS_MAP.values())
Q_and_A_classifications = llm_classify(
    dataframe=df,
    template=templates.QA_PROMPT_TEMPLATE,
    model=model,
    rails=rails,
    concurrency=10,
)["label"].tolist()

llm_classify |██████████| 22/22 (100.0%) | ⏳ 00:06<00:00 |  3.25it/s


In [16]:
Q_and_A_classifications

['correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'correct',
 'incorrect',
 'incorrect']

### create dataset for finetuning

In [17]:
finetune_df = pd.DataFrame.from_dict(parsed_response, orient='index', columns=['post', 'sentiment'])

In [18]:
def create_finetune_files(finetune_df):
    '''create both training and validation jsonl files for fine-tuning the model'''
    # Prepare to write to a .jsonl file
    formatted_examples = []
    for index, row in finetune_df.iterrows():
        # Structure the data as needed for fine-tuning
        formatted_example = {
            "messages": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": row.post},
                {"role": "assistant", "content": row.sentiment}
            ]
        }
        formatted_examples.append(formatted_example)
    # split into train and validation sets
    train = formatted_examples[:int(len(formatted_examples)*0.8)]
    validation = formatted_examples[int(len(formatted_examples)*0.8):]    
     # write each to a jsonl file
    with open('fine_tune_train_data.jsonl', 'w') as file:
        for example in train:
            file.write(json.dumps(example) + '\n')
    
    with open('fine_tune_validation_data.jsonl', 'w') as file:
        for example in validation:
            file.write(json.dumps(example) + '\n')

In [19]:
create_finetune_files(finetune_df)

# Finetune the model

In [20]:
# Upload the training file
training_file_response = client.files.create(
    file=open('fine_tune_train_data.jsonl', "rb"),
    purpose='fine-tune'
)
training_file_id = training_file_response.id

In [21]:
# Upload the validation file
validation_file_response = client.files.create(
    file=open('fine_tune_validation_data.jsonl', "rb"),
    purpose='fine-tune'
)
validation_file_id = validation_file_response.id

In [23]:
fine_tuning_info = client.fine_tuning.jobs.create(
  training_file=training_file_id, 
  model="gpt-3.5-turbo", 
  suffix="sentiment-small",
  validation_file=validation_file_id
)

In [36]:
for ft_job in client.fine_tuning.jobs.list(limit=5).data:
    print(ft_job.fine_tuned_model)

ft:gpt-3.5-turbo-0125:personal:sentiment-small:8zh5RxzN
ft:gpt-3.5-turbo-0613:personal:spam-classifier:8pLNGstw


In [37]:
ft_model = 'ft:gpt-3.5-turbo-0125:personal:sentiment-small:8zh5RxzN'

# evaluate the new model

In [40]:
new_df = pd.DataFrame(original_dataset, columns=['text', 'label'])
new_df['predicted_label'] = new_df['text'].apply(lambda x: classify_sentiment(x, model=ft_model)).apply(lambda x: x.strip().lower())

In [46]:
new_df

,text,label,predicted_label
0,I recently purchased the Galaxy Explorer drone...,positive,positive
1,Had dinner at The Green Terrace last night and...,negative,negative
2,"Lost my job, but at least I won't have to endu...",positive,positive
3,Your presentation was surprisingly good; I exp...,negative,negative
